In [1]:
# Move up a directory (until the root of the git repo),
# but only if this has not been done before.
if 'in_correct_dir' not in locals():
    %cd -q ..
in_correct_dir = True
%pwd

'/home/wout/pp/code'

In [1]:
import io
from pathlib import Path

import ipywidgets as W
from PIL import Image
from IPython.display import display

import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv("artifacts/results/mnist_default_assessor.csv")

In [17]:
df.y_true = df.y_true.replace({True: 1, False: 0})
df.y_pred = df.y_pred.replace({True: 1, False: 0})
df.is_correct = df.is_correct.replace({True: 1, False: 0})
df.head(5)

,index,y_true,y_pred,prediction,loss,is_correct
0,NaN,1,1,1.000000,1.192093e-07,1
1,NaN,1,1,0.999971,2.932592e-05,1
2,NaN,1,1,0.999442,5.577570e-04,1
3,NaN,1,1,0.999561,4.393231e-04,1
4,NaN,1,1,0.999960,4.029355e-05,1


In [18]:
df.describe()

,index,y_true,y_pred,prediction,loss,is_correct
count,0.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,NaN,0.978300,0.994300,0.981834,0.105461,0.975800
std,NaN,0.145709,0.075287,0.067732,0.684002,0.153677
min,NaN,0.000000,0.000000,0.017442,-0.000000,0.000000
25%,NaN,1.000000,1.000000,0.992989,0.000172,1.000000
50%,NaN,1.000000,1.000000,0.998875,0.001150,1.000000
75%,NaN,1.000000,1.000000,0.999829,0.007523,1.000000
max,NaN,1.000000,1.000000,1.000000,15.333239,1.000000


In [19]:
print(f"Num base mispredictions {(df['y_true'] == 0).sum() }")
pd.crosstab(df.y_true, df.y_pred)
# Make AUC
# Give it identity feature

Num base mispredictions 217


y_pred,0,1
y_true,,
0,16,201
1,41,9742
